In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv1(x), 2))
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x  # nn.functional.log_softmax 사용 제거


In [2]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, ConcatDataset

# 데이터 로딩 및 전처리
def load_mnist_data(train_dirs, test_dir):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    train_datasets = [datasets.ImageFolder(root=dir, transform=transform) for dir in train_dirs]
    train_dataset = ConcatDataset(train_datasets)
    test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    return train_loader, test_loader



In [3]:
import torch
import torch.optim as optim
import torch.nn.functional as F

def train_and_evaluate(model, train_loader, test_loader, epochs=10, device='cuda'):
    optimizer = optim.Adam(model.parameters())
    criterion = torch.nn.CrossEntropyLoss()

    model.to(device)

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            if batch_idx % 100 == 0:
                print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.4f}%)]\tLoss: {loss.item():.6f}')

        train_loss /= len(train_loader.dataset)
        train_accuracy = 100. * correct / total
        print(f'\nTraining set: Average loss: {train_loss:.4f}, Accuracy: {correct}/{total} ({train_accuracy:.0f}%)\n')

        # Evaluate on test data
        model.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += criterion(output, target).item()
                _, predicted = torch.max(output.data, 1)
                correct += (predicted == target).sum().item()

        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({test_accuracy:.4f}%)\n')

In [4]:
# 실행
train_dirs = ['C:/Users/koo/my_workspace/privacy_term_project/random_pertubation_image', 
             'C:/Users/koo/my_workspace/privacy_term_project/mnist_btm_hash/training' ]

test_dir = 'C:/Users/koo/my_workspace/privacy_term_project/mnist_btm_hash/testing'
model = SimpleCNN()
train_loader, test_loader = load_mnist_data(train_dirs, test_dir)
train_and_evaluate(model, train_loader, test_loader, epochs=10)
torch.save(model.state_dict(), 'mnist_cnn_by_ADT.pth') # 모델 저장

Train Epoch: 0 [0/62985 (0.0000%)]	Loss: 2.313155
Train Epoch: 0 [6400/62985 (10.1523%)]	Loss: 0.254351
Train Epoch: 0 [12800/62985 (20.3046%)]	Loss: 0.286899
Train Epoch: 0 [19200/62985 (30.4569%)]	Loss: 0.084228
Train Epoch: 0 [25600/62985 (40.6091%)]	Loss: 0.092451
Train Epoch: 0 [32000/62985 (50.7614%)]	Loss: 0.072790
Train Epoch: 0 [38400/62985 (60.9137%)]	Loss: 0.031163
Train Epoch: 0 [44800/62985 (71.0660%)]	Loss: 0.070075
Train Epoch: 0 [51200/62985 (81.2183%)]	Loss: 0.033293
Train Epoch: 0 [57600/62985 (91.3706%)]	Loss: 0.132110

Training set: Average loss: 0.0036, Accuracy: 58781/62985 (93%)

Test set: Average loss: 0.0009, Accuracy: 9827/10000 (98.2700%)

Train Epoch: 1 [0/62985 (0.0000%)]	Loss: 0.080494
Train Epoch: 1 [6400/62985 (10.1523%)]	Loss: 0.034822
Train Epoch: 1 [12800/62985 (20.3046%)]	Loss: 0.135736
Train Epoch: 1 [19200/62985 (30.4569%)]	Loss: 0.128781
Train Epoch: 1 [25600/62985 (40.6091%)]	Loss: 0.029058
Train Epoch: 1 [32000/62985 (50.7614%)]	Loss: 0.042143
T